In [1]:
import pandas as pd
df = pd.read_csv("train.csv")

In [2]:
pd.set_option('display.max_colwidth', 2000)

In [21]:
df.head()

,itemid,title,Category,image_path
0,307504,nyx sex bomb pallete natural palette,0,beauty_image/6b2e9cbb279ac95703348368aa65da09.jpg
1,461203,etude house precious mineral any cushion pearl aura puff,1,beauty_image/20450222d857c9571ba8fa23bdedc8c9.jpg
2,3592295,milani rose powder blush,2,beauty_image/6a5962bed605a3dd6604ca3a4278a4f9.jpg
3,4460167,etude house baby sweet sugar powder,3,beauty_image/56987ae186e8a8e71fcc5a261ca485da.jpg
4,5853995,bedak revlon color stay aqua mineral make up,3,beauty_image/9c6968066ebab57588c2f757a240d8b9.jpg


In [3]:
title = df['title'].tolist()
category = df['Category'].tolist()

In [31]:
title

['nyx sex bomb pallete natural palette',
 'etude house precious mineral any cushion pearl aura puff',
 'milani rose powder blush',
 'etude house baby sweet sugar powder',
 'bedak revlon color stay aqua mineral make up',
 'dr pure whitening cream',
 'chanel powder blush malice',
 'snail white cream original 100',
 'sunprise all proof spf 50',
 'eyebrow powder nyx satuan rp 15.000 pc',
 'monistat chafing relief gel',
 'milani rose powder blush tea',
 'the balm meet matte trimony',
 'laneige water base cc cream spf36 pa',
 'giordani gold age defying compact foundation dengan spf 15 perfect cover',
 'the body shop refill moisture white perfect foundation',
 'lancome blush subtil long lasting powder blusher colour veil buildable intensity',
 'missha line friends magic cushion moisture 2 refills',
 'elf acne fighting foundation beige',
 'cream dr biru original',
 'city color cream concealer contour palette',
 'etude pink vital water special trial kit isi 4',
 'etude magic any cushion refill 

In [8]:
a=[]
for i in category:
    if i not in a:
        a.append(i)
max(a)
a

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57]

In [12]:
count=[]
for i in a:
    count.append(category.count(i))
count
        

[3772,
 28670,
 11544,
 81250,
 42688,
 55279,
 2047,
 11668,
 6048,
 8106,
 1077,
 4108,
 21782,
 2982,
 2735,
 599,
 2228,
 2765,
 56598,
 13493,
 19978,
 10483,
 15225,
 1671,
 4251,
 33922,
 33845,
 16199,
 6493,
 3338,
 1441,
 27836,
 29754,
 4821,
 14718,
 30590,
 1028,
 2272,
 4703,
 697,
 327,
 19312,
 10502,
 6147,
 920,
 2195,
 684,
 945,
 416,
 573,
 282,
 404,
 117,
 418,
 300,
 151,
 170,
 48]

In [19]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = len(title))
all_data_features = vectorizer.fit_transform(title)

In [51]:
print(all_data_features[1])

  (0, 3677)	0.24816506740996583
  (0, 4654)	0.2570942426210359
  (0, 7356)	0.31862131552595163
  (0, 6198)	0.26832281793998813
  (0, 1587)	0.3976059309134311
  (0, 3099)	0.20647284387172885
  (0, 6988)	0.420674432606434
  (0, 1706)	0.42603502972458734
  (0, 7443)	0.3736383760249856


In [17]:
vec_sample = all_data_features[1].toarray()
print(all_data_features[1])
print(vec_sample)
print(len(vec_sample))

  (0, 60004)	1
  (0, 17965)	1
  (0, 57393)	1
  (0, 27337)	1
  (0, 17187)	1
  (0, 51482)	1
  (0, 59385)	1
  (0, 39405)	1
  (0, 32495)	1
[[0 0 0 ... 0 0 0]]
1


In [20]:
all_data_features.getnnz()

5838779

In [21]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(all_data_features, category, test_size=0.25, random_state=20)

In [22]:
X_train.getnnz()

4378018

In [23]:
from sklearn import svm
clf=svm.LinearSVC()
clf.fit(X_train,y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [24]:
from sklearn.metrics import accuracy_score
# predict( ) will do the model prediction, predict y based on the input x
predict_y = clf.predict(X_test)
print ('testing acc is %f' %accuracy_score(predict_y, y_test))

testing acc is 0.719047


In [25]:
y_test

[41,
 20,
 32,
 4,
 51,
 19,
 26,
 31,
 5,
 3,
 20,
 18,
 21,
 12,
 41,
 35,
 18,
 28,
 3,
 25,
 31,
 5,
 20,
 18,
 23,
 26,
 35,
 19,
 3,
 3,
 3,
 31,
 3,
 5,
 32,
 25,
 3,
 1,
 31,
 22,
 1,
 20,
 4,
 5,
 22,
 31,
 35,
 3,
 3,
 26,
 25,
 32,
 26,
 10,
 18,
 8,
 33,
 21,
 20,
 25,
 3,
 26,
 5,
 26,
 5,
 3,
 26,
 35,
 19,
 3,
 19,
 41,
 3,
 3,
 1,
 4,
 5,
 4,
 3,
 20,
 20,
 5,
 26,
 18,
 23,
 41,
 1,
 3,
 24,
 7,
 25,
 4,
 5,
 11,
 34,
 5,
 25,
 41,
 25,
 4,
 1,
 13,
 32,
 22,
 35,
 18,
 18,
 5,
 9,
 7,
 8,
 18,
 26,
 1,
 25,
 27,
 43,
 20,
 42,
 29,
 25,
 12,
 46,
 35,
 26,
 31,
 30,
 4,
 18,
 25,
 18,
 34,
 26,
 18,
 3,
 5,
 4,
 24,
 4,
 25,
 41,
 26,
 18,
 5,
 18,
 7,
 3,
 12,
 44,
 27,
 32,
 5,
 19,
 22,
 37,
 35,
 5,
 41,
 41,
 53,
 3,
 44,
 12,
 25,
 35,
 25,
 3,
 5,
 5,
 4,
 12,
 26,
 1,
 33,
 5,
 26,
 5,
 3,
 32,
 32,
 13,
 18,
 3,
 3,
 4,
 19,
 1,
 12,
 4,
 19,
 5,
 45,
 7,
 7,
 18,
 5,
 11,
 26,
 20,
 1,
 3,
 18,
 28,
 53,
 27,
 31,
 18,
 18,
 32,
 3,
 3,
 21,
 5,
 32,
 5,
 43

In [26]:
test_data=pd.read_csv('test.csv')

In [31]:
test_data.head()

,itemid,title,image_path
0,370855998,flormar 7 white cream bb spf 30 40ml,beauty_image/1588591395c5a254bab84042005f2a9f.jpg
1,637234604,maybelline clear smooth all in one bb cream spf 21,beauty_image/920985ed9587ea20f58686ea74e20f93.jpg
2,690282890,murah innisfree eco natural green tea bb cream spf25 pa eksklusif,beauty_image/90b40e5710f54352b243fcfb0f5d1d7f.jpg
3,930913462,loreal white perfect day cream spf 17 pa whitening+even tone murah,beauty_image/289c668ef3d70e1d929d602d52d5d78a.jpg
4,1039280071,hada labo cc cream ultimate anti aging spf 35 pa 25gr pilih warna,beauty_image/d5b3e652c5822d2306f4560488ec30c6.jpg


In [32]:
tes=test_data['title'].tolist()
tes=vectorizer.fit_transform(tes)

In [33]:
pred=clf.predict(tes)

ValueError: X has 35754 features per sample; expecting 80055

In [34]:
pred

NameError: name 'pred' is not defined

In [68]:
subm=pd.read_csv('data_info_val_sample_submission.csv')

In [44]:
subm['Category']=pred

In [46]:
subm.head()

,itemid,Category
0,370855998,1
1,637234604,19
2,690282890,3
3,930913462,3
4,1039280071,26


In [47]:
subm.to_csv('Submission.csv', index=False)

In [2]:
tes.getnnz()

NameError: name 'tes' is not defined